In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
names = ['userid','time','extype','examount','mark']
bank_detail_train = pd.read_csv("../../pcredit/train/bank_detail_train.txt",header=None)
bank_detail_test = pd.read_csv("../../pcredit/test/bank_detail_test.txt",header=None)

bank_detail = pd.concat([bank_detail_train,bank_detail_test])
bank_detail.columns = names
bank_detail.head()

,userid,time,extype,examount,mark
0,6965,5894316387,0,13.756664,0
1,6965,5894321388,1,13.756664,0
2,6965,5897553564,0,14.449810,0
3,6965,5897563463,1,10.527763,0
4,6965,5897564598,1,13.651303,0


In [7]:
import warnings
warnings.filterwarnings('ignore')  #  忽略警告
bank_type_amount = bank_detail[['userid','extype','examount']]
bank_type_amount['bank_type_amount_n'] = 1
bank_type_amount.head()

,userid,extype,examount,bank_type_amount_n
0,6965,0,13.756664,1
1,6965,1,13.756664,1
2,6965,0,14.449810,1
3,6965,1,10.527763,1
4,6965,1,13.651303,1


In [9]:
bank_type_amount_nsum = bank_type_amount.groupby(['userid','extype'])['bank_type_amount_n'].sum()
bank_type_amount_nsum.head()

userid  extype
1       0          35
        1          51
3       0         172
        1         507
4       0          96
Name: bank_type_amount_n, dtype: int64

In [11]:
bank_type_amount_sum = bank_type_amount_nsum.unstack()
bank_type_amount_sum.columns = ["extype##0","extype##1"]
bank_type_amount_sum.head()

,extype##0,extype##1
userid,,
1,35.0,51.0
3,172.0,507.0
4,96.0,195.0
9,13.0,11.0
10,141.0,183.0


In [12]:
df = pd.pivot_table(bank_detail,index=['userid','extype'],values=['examount'],aggfunc=np.mean)
df.head()

examount
userid extype           
1      0       13.734079
       1       13.255397
3      0       13.249264
       1        9.834236
4      0       12.128566

In [13]:
dfun = df.unstack()
dfun['sub'] = dfun['examount'][0]-dfun['examount'][1]
dfun.head()

examount                  sub
extype          0          1          
userid                                
1       13.734079  13.255397  0.478681
3       13.249264   9.834236  3.415028
4       12.128566  10.920132  1.208435
9       14.371789  12.867697  1.504092
10      12.720866  12.296680  0.424186

In [14]:
dfun.columns = ['examount#0','examount#1','sub']
dfun.head()

,examount#0,examount#1,sub
userid,,,
1,13.734079,13.255397,0.478681
3,13.249264,9.834236,3.415028
4,12.128566,10.920132,1.208435
9,14.371789,12.867697,1.504092
10,12.720866,12.296680,0.424186


In [18]:
dfun.describe()

,examount#0,examount#1,sub
count,10002.000000,9989.000000,9988.000000
mean,12.720775,11.475582,1.247743
std,1.117487,1.065881,0.773776
min,2.344542,2.344542,-5.777228
25%,12.029853,10.766973,0.790818
50%,12.710153,11.425952,1.217663
75%,13.409364,12.130727,1.663185
max,18.112103,16.863589,5.408413


In [19]:
dfun.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10003 entries, 1 to 69094
Data columns (total 3 columns):
examount#0    10002 non-null float64
examount#1    9989 non-null float64
sub           9988 non-null float64
dtypes: float64(3)
memory usage: 312.6 KB


In [21]:
dfun = dfun.dropna()
dfun.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9988 entries, 1 to 69094
Data columns (total 3 columns):
examount#0    9988 non-null float64
examount#1    9988 non-null float64
sub           9988 non-null float64
dtypes: float64(3)
memory usage: 312.1 KB


In [22]:
#  对 new_bill_data_mean 的所有列数据进行标签编码
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cols = dfun.columns
def transforLabel(datas,cols):
    for col in cols:
        data = datas[col]
        le.fit(data)
        datas[col] = le.transform(data)
    return datas
bank_label_data = transforLabel(dfun,cols)
bank_label_data.head()

,examount#0,examount#1,sub
userid,,,
1,8296,9452,1227
3,6979,487,9910
4,2817,3042,4926
9,9330,8994,6687
10,5030,7922,1094


In [1]:
''' #  不构建哑变量
#  构建哑变量
#print bill_label_data.head()
cols = bank_label_data.columns
def dummyData(datas,cols):
    for col in cols:
        s = pd.cut(datas[col],20)
        d = pd.get_dummies(s)
        d.columns = ["{}#{}".format(col,i) for i in range(20)]
        datas.drop(col,axis = 1,inplace = True)
        datas = datas.join(d)
    return datas
bank_dummy_data = dummyData(bank_label_data.copy(),cols)
bank_dummy_data.head()
'''

' #  \xe4\xb8\x8d\xe6\x9e\x84\xe5\xbb\xba\xe5\x93\x91\xe5\x8f\x98\xe9\x87\x8f\n#  \xe6\x9e\x84\xe5\xbb\xba\xe5\x93\x91\xe5\x8f\x98\xe9\x87\x8f\n#print bill_label_data.head()\ncols = bank_label_data.columns\ndef dummyData(datas,cols):\n    for col in cols:\n        s = pd.cut(datas[col],20)\n        d = pd.get_dummies(s)\n        d.columns = ["{}#{}".format(col,i) for i in range(20)]\n        datas.drop(col,axis = 1,inplace = True)\n        datas = datas.join(d)\n    return datas\nbank_dummy_data = dummyData(bank_label_data.copy(),cols)\nbank_dummy_data.head()\n'

In [15]:
dfun = dfun.join(bank_type_amount_sum)
dfun['bank_tag'] = 1
dfun.head()

,examount#0,examount#1,sub,extype##0,extype##1,bank_tag
userid,,,,,,
1,13.734079,13.255397,0.478681,35.0,51.0,1
3,13.249264,9.834236,3.415028,172.0,507.0,1
4,12.128566,10.920132,1.208435,96.0,195.0,1
9,14.371789,12.867697,1.504092,13.0,11.0,1
10,12.720866,12.296680,0.424186,141.0,183.0,1


In [16]:
dfun.to_csv('../data/train/bank_data.csv')